In [ ]:
import numpy as np
import torch
import torch.nn as nn

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=256, num_layers=2,bidirectional=False,dropout_rate=0.5,fc_units = 512):
        super().__init__()
        self.bidirectional = bidirectional
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers,dropout = dropout_rate,batch_first=True,bidirectional=bidirectional)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(hidden_size, fc_units)
        self.fc2 = nn.Linear(fc_units, fc_units//2)
        self.fc3 = nn.Linear(fc_units//2, fc_units//4)
        self.fc4 = nn.Linear(fc_units//4, 1)

        if(self.bidirectional == True):
            self.fc1 = nn.Linear(hidden_size*2, fc_units*2)
            self.fc2 = nn.Linear(fc_units*2, fc_units)
            self.fc3 = nn.Linear(fc_units, fc_units//2)
            self.fc4 = nn.Linear(fc_units//2, 1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        if(self.bidirectional == True):
            direction = 2
        else:
            direction = 1

        h0 = torch.zeros(direction*self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(direction*self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _  = self.lstm(x, (h0, c0))
        out_fc1 = self.fc1(out[:, -1, :])  # Use only the last output
        out_fc1 = self.relu(out_fc1)
        out_fc2 = self.fc2(self.dropout(out_fc1))
        out_fc2 = self.relu(out_fc2)
        out_fc3 = self.fc3(self.dropout(out_fc2))
        out_fc3 = self.relu(out_fc3)
        out_fc4 = self.fc4(self.dropout(out_fc3))
        output = self.sigmoid(out_fc4)
        output = output.view((len(x)))
        return output

In [ ]:
class GRU(nn.Module):
    def __init__(self, input_size=1, hidden_size=256, num_layers=2,bidirectional=False,dropout_rate = 0.5,fc_units = 512):
        super().__init__()
        self.bidirectional = bidirectional
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size = input_size, hidden_size = hidden_size, num_layers=num_layers, dropout = dropout_rate,batch_first=True,bidirectional=bidirectional)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(hidden_size, fc_units)
        self.fc2 = nn.Linear(fc_units, fc_units//2)
        self.fc3 = nn.Linear(fc_units//2, fc_units//4)
        self.fc4 = nn.Linear(fc_units//4, 1)

        if(self.bidirectional == True):
            self.fc1 = nn.Linear(hidden_size*2, fc_units*2)
            self.fc2 = nn.Linear(fc_units*2, fc_units)
            self.fc3 = nn.Linear(fc_units, fc_units//2)
            self.fc4 = nn.Linear(fc_units//2, 1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        if(self.bidirectional == True):
            direction = 2
        else:
            direction = 1

        h0 = torch.zeros(direction*self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.gru(x, h0)
        out_fc1 = self.fc1(out[:, -1, :])  # Use only the last output
        out_fc1 = self.relu(out_fc1)
        out_fc2 = self.fc2(self.dropout(out_fc1))
        out_fc2 = self.relu(out_fc2)
        out_fc3 = self.fc3(self.dropout(out_fc2))
        out_fc3 = self.relu(out_fc3)
        output = self.fc4(self.dropout(out_fc3))
        output = self.sigmoid(output)
        return output

In [ ]:
class CNN1D(nn.Module):
    def __init__(self, input_size=1,sequence_length=800,fc_units=512,dropout_rate= 0.4):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=60, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=60, out_channels=40, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=40, out_channels=20, kernel_size=3)
        self.conv4 = nn.Conv1d(in_channels=20, out_channels=10, kernel_size=3)

        self.pool = nn.MaxPool1d(kernel_size=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        # Calculate input size for the fully connected layer
        conv_output_size = self._get_conv_output_size(input_size, sequence_length)
        self.fc1 = nn.Linear(conv_output_size, fc_units)

        self.fc2 = nn.Linear(fc_units, fc_units//2)
        self.fc3 = nn.Linear(fc_units//2, fc_units//4)
        self.fc4 = nn.Linear(fc_units//4, 1)

        self.sigmoid = nn.Sigmoid()

    def _get_conv_output_size(self, input_size, sequence_length):
        # Calculate the output size after passing through the convolutional layers and pooling layer
        x = torch.randn(256, input_size, sequence_length)
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))

        return x.size(1) * x.size(2)

    def forward(self, x):
        # Forward pass through convolutional layers
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))

        # Flatten the output
        x = x.view(x.size(0), -1)

        # Forward pass through fully connected layers
        x = self.relu(self.fc1(x))
        x = self.fc2(self.dropout(x))
        x = self.relu(x)
        x = self.fc3(self.dropout(x))
        x = self.relu(x)
        x = self.fc4(self.dropout(x))

        # Apply sigmoid activation function
        x = self.sigmoid(x)
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=8, num_encoder_layers=6, dim_feedforward=256,fc_units=512, dropout_rate=0.3, max_len=800):
        super(Transformer, self).__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_len)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout_rate,batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_encoder_layers)
        self.d_model = d_model
        self.fc1 = nn.Linear(d_model, fc_units)
        self.fc2 = nn.Linear(fc_units, fc_units//2)
        self.fc3 = nn.Linear(fc_units//2, fc_units//4)
        self.fc4 = nn.Linear(fc_units//4, 1)
        self.gelu = nn.GELU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = torch.mean(x,dim=1)  # Global average pooling
        x = self.fc1(x)
        x = self.gelu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.gelu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.gelu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        output = self.sigmoid(x)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=800):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:,:x.size(1), :]